Open AI (https://developers.openai.com/api/docs/guides/embeddings/)
- Open AI 의 text-embedding-3-large 모델은 차원을 줄여도 의미 표현이 잘 유지되도록 학습됨

# Embedding

- 특정 단어, 문장, 문서 를 특정 차원으로 표현
<img src="https://d2908q01vomqb2.cloudfront.net/887309d048beef83ad3eabf2a79a64a389ab1c9f/2023/07/25/dbblog-3423-image001.png"/>

- 두 텍스트 사이의 관련성을 측정하는 데 사용할 수 있는 텍스트의 숫자 표현  
- 검색, 클러스터링, 추천, 이상 탐지 및 분류 작업에 유용

    - 검색 (쿼리 문자열과의 관련성을 기준으로 결과 순위 지정)  
    - 클러스터링 (텍스트 문자열이 유사성을 기준으로 그룹화)  
    - 추천 (관련 문자열이 포함된 항목을 추천)  
    - 이상 탐지 (관련성이 거의 없는 이상값이 식별되는 경우)  
    - 다양성 측정 (유사성 분포를 분석)
    - 분류 (텍스트 문자열이 가장 유사한 레이블로 분류)
  
- 거리가 작을수록 관련성이 높음을 나타내고, 거리가 멀면 관련성이 낮다는 것을 나타낸다.  

In [1]:
from dotenv import load_dotenv, find_dotenv
# .env 파일 가져오기
load_dotenv(find_dotenv())

True

In [2]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small" # 이 모델을 사용하지 않는다면 개발자가 직접 코드 구현
)

print(response.data[0].embedding)

[0.005126522853970528, 0.01728670485317707, -0.018700435757637024, -0.018574459478259087, -0.04722699895501137, -0.03029022552073002, 0.027686722576618195, 0.0036358069628477097, 0.011218861676752567, 0.006403779145330191, -0.0017304201610386372, 0.015816984698176384, -0.0013078757328912616, -0.007803512271493673, 0.059852588921785355, 0.05033440515398979, -0.027504757046699524, 0.00991710927337408, -0.04042429476976395, 0.04999846965074539, -0.00039804913103580475, 0.030234236270189285, -0.01373138278722763, 0.0328657366335392, 0.01730070263147354, 0.016810795292258263, -0.001747041940689087, 0.020422106608748436, 0.04076023027300835, -0.03776480257511139, -0.026133017614483833, -0.04997047409415245, 0.024187389761209488, -0.05526146665215492, -0.03230584040284157, 0.04235592484474182, 0.06461168080568314, 0.014683200977742672, -0.015635019168257713, -0.04134811833500862, 0.02217177301645279, 0.007362596690654755, 0.04490343853831291, 0.007089648395776749, -0.02406141348183155, 0.0524

In [ ]:
# text-embedding-3-small : vector = 1536 모델

len(response.data[0].embedding)

1536

### 벡터 유사도

- **벡터 유사도**는  임베딩으로 구한 벡터 숫자 집합이 나타내는 항목이 서로 얼마나 유사한가?


<img src=https://cdn.sanity.io/images/vr8gru94/production/5a5ba7e0971f7b6dc4697732fa8adc59a46b6d8d-338x357.png width=200 />

$\cos(\theta) = \frac{A \cdot B}{||A|| \times ||B||}$

- +1 에 가까울 수록 의미가 비슷
- 0 에 가까울 수록 관계 없음
- -1 에 가까울 수록 반대방향(의미가 반대이거나, 완전 다름)

In [4]:
import numpy as np
from numpy import dot 
import pandas as pd
from numpy.linalg import norm

In [ ]:
# 코사인 유사도 함수 (a와 b 사이의 거리 구하기)

def cosine_similarity(A,B):
    return dot(A, B) / (norm(A) * norm(B))

In [6]:
# 유사도 구하기
vec1 = np.array([0,1,1,1])
vec2 = np.array([1,0,1,1])
vec3 = np.array([2,0,2,2])

print(f"vec1과 vec2의 유사도 : {cosine_similarity(vec1,vec2)}")
print(f"vec1과 vec3의 유사도 : {cosine_similarity(vec1,vec3)}")
print(f"vec2과 vec3의 유사도 : {cosine_similarity(vec2,vec3)}")

vec1과 vec2의 유사도 : 0.6666666666666667
vec1과 vec3의 유사도 : 0.6666666666666667
vec2과 vec3의 유사도 : 1.0000000000000002


In [7]:
# 문장의 유사도 구하기(vector)

data = [
    '저는 배가 고파요',
    '저기 배가 지나가네요',
    '굶어서 허기가 지네요',
    '허기 워기라는 게임이 있는데 즐거워',
    '스팀에서 재밌는거 해야지',
    '스팀에어프라이어로 연어구이 해 먹을거야',
]

df = pd.DataFrame(data, columns=['text'])
df

,text
0,저는 배가 고파요
1,저기 배가 지나가네요
2,굶어서 허기가 지네요
3,허기 워기라는 게임이 있는데 즐거워
4,스팀에서 재밌는거 해야지
5,스팀에어프라이어로 연어구이 해 먹을거야


In [ ]:
# text를 embedding 숫자로 만드는 함수(성공적)

def embedding_func(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )

    return response.data[0].embedding

In [12]:
embedding_func("저는 배가 고파요")

[0.017315642908215523,
 -0.007798387203365564,
 -0.029025429859757423,
 0.007520047482103109,
 0.0011255656136199832,
 -0.05871497094631195,
 -0.03381091356277466,
 0.03363512083888054,
 -0.04672195762395859,
 -0.015460046008229256,
 -0.020645948126912117,
 0.051409780979156494,
 -0.0011121368734166026,
 -0.003862569807097316,
 -0.01698358729481697,
 0.0023280407767742872,
 -0.0582461878657341,
 0.00046420423313975334,
 0.005034525413066149,
 -0.03273662179708481,
 -0.02449386939406395,
 -0.015733502805233,
 -0.023224251344799995,
 -0.006821757648140192,
 0.0020460388623178005,
 -0.0004272754304111004,
 0.03951443359255791,
 -0.03371325135231018,
 0.0037185170222073793,
 -0.011621891520917416,
 0.009859075769782066,
 -0.04765952378511429,
 0.01810671202838421,
 -0.04844082519412041,
 -0.019024744629859924,
 -0.038791727274656296,
 0.01170978881418705,
 -0.03769790008664131,
 -0.004265429452061653,
 0.0012134622083976865,
 -0.029142625629901886,
 0.0015296461060643196,
 -0.0178918540477

In [13]:
# pandas2.ipynb 마지막에서 배운 내용
df['embedding'] = df['text'].apply(embedding_func)
df

,text,embedding
0,저는 배가 고파요,"[0.017315642908215523, -0.007798387203365564, ..."
1,저기 배가 지나가네요,"[0.039444465190172195, 0.00010226095764664933,..."
2,굶어서 허기가 지네요,"[0.04162225499749184, 0.025161150842905045, -0..."
3,허기 워기라는 게임이 있는데 즐거워,"[0.007540854159742594, -0.003866263898089528, ..."
4,스팀에서 재밌는거 해야지,"[0.007323416415601969, 0.054135024547576904, -..."
5,스팀에어프라이어로 연어구이 해 먹을거야,"[0.04879317805171013, -0.003386686323210597, 0..."


In [14]:
def return_answer_candidate(df, query):
    # 사용자가 입력한 문장(query) 의 임베딩 구하기
    query_embedding = embedding_func(query)

    df['similarity'] = df['embedding'].apply(lambda x:cosine_similarity(np.array(x), np.array(query_embedding)))

    top_three = df.sort_values('similarity', ascending=False).head(3)

    return top_three

In [16]:
# 코사인 유사도를 이용해 사용자가 입력한 문장과 유사한 의미의 문장 추출

user_input = "배에서 꼬르륵 소리가 나네"
top_three_result = return_answer_candidate(df, user_input)
top_three_result

,text,embedding,similarity
1,저기 배가 지나가네요,"[0.039444465190172195, 0.00010226095764664933,...",0.311882
4,스팀에서 재밌는거 해야지,"[0.007323416415601969, 0.054135024547576904, -...",0.276534
2,굶어서 허기가 지네요,"[0.04162225499749184, 0.025161150842905045, -0...",0.272694


## Amazon 고급 음식 리뷰를 이용한 Text Search

- 원 데이터(https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews)에는 2012년 10월까지 Amazon 사용자가 남긴 총 568,454개의 음식 리뷰가 포함되어 있음.
- 이 중 가장 최근 리뷰 1,000개로 구성된 이 데이터 세트의 하위 세트 사용
- 리뷰는 영어로 작성되어 있으며 긍정 or 부정의 리뷰로 구성
- 각 리뷰에는 ProductId, UserId, 점수, 리뷰 제목(요약) 및 리뷰 본문(텍스트) 컬럼 존재
- 리뷰 요약과 리뷰 텍스트를 하나의 텍스트로 결합 후 결합된 텍스트를 인코딩하고 단일 벡터 임베딩을 출력

In [ ]:

# embedding 값 추출 함수
# embedding_model = "text-embedding-ada-002"
# def get_embedding(text: str, model=embedding_model):
#     text = text.replace("\n", " ")  # 성능에 부정적 영향을 줄 수 있는 \n 제거
#     response = client.embeddings.create(input=[text], model=model)
#     return response.data[0].embedding


# "combined" 컬럼(리뷰 제목 + 본문)에 대해 임베딩을 생성하여 "embedding" 컬럼에 저장
# amazone_df["embedding"] = amazone_df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
#  임베딩이 포함된 데이터프레임을 CSV 파일로 저장
# amazone_df.to_csv("data/fine_food_reviews_with_embeddings_1k.csv")

In [17]:
df = pd.read_csv("./data/fine_food_reviews_with_embeddings_1k.csv", index_col=0)

# "[ embedding 숫자 ]"

df['embedding'] = df['embedding'].apply(eval).to_list()

df.head(2)

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
0,B003XPF9BO,A3R7JR3FMEBXQB,5,이런 식으로 어디서 시작하고 멈추는가,시카고 가족에게 가져 오기 위해 일부를 절약하고 싶었지만 노스 캐롤라이나 가족은 4...,Title: 이런 식으로 어디서 시작하고 멈추는가; Content: 시카고 가족에게...,87,"[-0.0022805905900895596, 0.0007801640313118696..."
1,B003JK537S,A3JBPC3WFUT5ZP,1,조각으로 도착했습니다,상자를 열었을 때 전혀 기뻐하지 않습니다. 대부분의 고리가 조각으로 부러졌습니다.,Title: 조각으로 도착했습니다; Content: 상자를 열었을 때 전혀 기뻐하지...,55,"[0.05709123983979225, 0.005073584616184235, -0..."


In [18]:
# embedding 값 활용 (1) 검색
# 활용1) 특정 리뷰와 유사한 리뷰 찾기

def search_reviews(df, query, top_n=3):
    # 사용자가 입력한 문장(query) 의 임베딩 구하기
    query_embedding = embedding_func(query)

    df['similarity'] = df['embedding'].apply(lambda x:cosine_similarity(np.array(x), np.array(query_embedding)))

    res = df.sort_values('similarity', ascending=False).head(top_n)

    return res

In [19]:
result = search_reviews(df, '맛있는 콩', top_n=2)
result

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding,similarity
771,B002GWHBU2,A2NUQSB42RKBFA,5,자메이카 푸른 콩,로스팅하기위한 우수한 커피 콩 우리 가족은 더 많은 양의 맛을 위해 5 파운드를 더...,Title: 자메이카 푸른 콩; Content: 로스팅하기위한 우수한 커피 콩 우리...,112,"[0.0030990454833954573, 0.03926049545407295, -...",0.520019
88,B000E3ZFDU,A1PQDL14230X6U,5,맛있는,나는이 흰 콩 조미료를 즐긴다 그것은 콩에 풍부한 맛을줍니다. 나는 법에있는 어머니...,Title: 맛있는; Content: 나는이 흰 콩 조미료를 즐긴다 그것은 콩에 풍...,181,"[0.04669349640607834, 0.03440799564123154, -0....",0.519946
